# Strassen Multiplication
by Miles Milosevich



## History

## Algorithm

Let us consider the "Divide and Conquer" strategy for matrix multiplication. In this, matrices are recursively divided into smaller quadrants, or block matrices, until only 2x2 matrices remain, which are multiplied. 

$$
\begin{bmatrix} 
C_{00} & C_{01} \\
C_{10} & C_{11} 
\end{bmatrix}
=
\begin{bmatrix} 
A_{00} & A_{01} \\
A_{10} & A_{11} 
\end{bmatrix}
\cdot
\begin{bmatrix} 
B_{00} & B_{01} \\
B_{10} & B_{11} 
\end{bmatrix}
$$

The formula for calculating each block of $C$:
$$
\begin{align*}
C_{00} &= A_{00} \cdot B_{00} + A_{01} \cdot B_{10} \\
C_{01} &= A_{00} \cdot B_{01} + A_{01} \cdot B_{11} \\
C_{10} &= A_{10} \cdot B_{00} + A_{11} \cdot B_{10} \\
C_{11} &= A_{10} \cdot B_{01} + A_{11} \cdot B_{11}
\end{align*}
$$

For recursive block matrix multiplication, matrices are padded until they have a dim size of a power of 2; as such, matrices can be subdivided to be entirely composed of 2x2 block matrices. This padding can be removed after multiplication.

#### Operations and Complexity
- **Number of Multiplications**: $ 8n^{\log_2 4} $ where $n$ is the dimension of the matrix, assuming no reductions or further optimizations.
- **Time Complexity**: $ T(n) = 8T\left(\frac{n}{2}\right) + O(n^2) $

The matrix multiplication operation is expensive, motivating the creation of an algorithm using fewer matrix multiplication operations. Strassen's algorithm implements this; in the above Naive approach, 8 multiplication operations are used, while is Strassen's below, only 7 are used.

$$
\begin{bmatrix} 
C_{00} & C_{01} \\
C_{10} & C_{11} 
\end{bmatrix}
=
\begin{bmatrix}
M_1 + M_4 - M_5 + M_7 & M_3 + M_5 \\
M_2 + M_4 & M_1 - M_2 + M_3 + M_6
\end{bmatrix}
$$

Where the $M_i$ terms are computed as:
$$
\begin{align*}
M_1 &= (A_{00} + A_{11}) \cdot (B_{00} + B_{11}) \\
M_2 &= (A_{10} + A_{11}) \cdot B_{00} \\
M_3 &= A_{00} \cdot (B_{01} - B_{11}) \\
M_4 &= A_{11} \cdot (B_{10} - B_{00}) \\
M_5 &= (A_{00} + A_{01}) \cdot B_{11} \\
M_6 &= (A_{10} - A_{00}) \cdot (B_{00} + B_{01}) \\
M_7 &= (A_{01} - A_{11}) \cdot (B_{10} + B_{11})
\end{align*}
$$

#### Operations and Complexity
- **Number of Multiplications**: $7^{\log_2(n/t)} $
- **Time Complexity**: $ T(n) = 7T\left(\frac{n}{2}\right) + O(n^2) $


## Implementation

Strassen's algorithm is known for only being beneficial in high-dimensional matrices of at least 512. In order process multiplication at this dimensionality, multiprocessing is implemented using the **multiprocess** library, a multiprocessing library capable of serializing constructs the builtin **multiprocessing** cannot, such as lambda functions and complex structures.

For full code, see BlockMMultFactory (an object for creating matrix multiplication operators) for implementation details regarding multiprocessing and generalized matrix multiplication

In [1]:
from blockmmult import BlockMMultFactory
import numpy as np


def create_strassen_callable(threshold=1, max_thread_depth=2):
    
    #select A and B quadrants for recursive block multiplication calls
    scatter_targets = [
        ((lambda A, B : A[0][0] + A[1][1]), (lambda A, B : B[0][0] + B[1][1])),
        ((lambda A, B : A[1][0] + A[1][1]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][1] - B[1][1])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][0] - B[0][0])),
        ((lambda A, B : A[0][0] + A[0][1]), (lambda A, B : B[1][1])),
        ((lambda A, B : A[1][0] - A[0][0]), (lambda A, B : B[0][0] + B[0][1])),
        ((lambda A, B : A[0][1] - A[1][1]), (lambda A, B : B[1][0] + B[1][1])),
    ]
    
    # how will results from the above be summated?
    gather_tasks = [
        (
        #C11 & C12
            (lambda M : M[0] + M[3] - M[4] + M[6]),
            (lambda M : M[2] + M[4]),
        ),(
        #C21 & C22
            (lambda M : M[1] + M[3]),
            (lambda M : M[0] - M[1] + M[2] + M[5]),
        )
    ]
    def strassen_mult_calls(n, threshold):
        return 7 ** (np.log2(n / threshold))
        
    return BlockMMultFactory(scatter_targets, gather_tasks, threshold, max_thread_depth, strassen_mult_calls)

def create_naive_callable(threshold=1, max_thread_depth=2):

    scatter_targets = [
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[0][1]), (lambda A, B : B[1][0])),
        ((lambda A, B : A[0][0]), (lambda A, B : B[0][1])),
        ((lambda A, B : A[0][1]), (lambda A, B : B[1][1])),
        ((lambda A, B : A[1][0]), (lambda A, B : B[0][0])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][0])),
        ((lambda A, B : A[1][0]), (lambda A, B : B[0][1])),
        ((lambda A, B : A[1][1]), (lambda A, B : B[1][1]))
    ]
    gather_tasks = [
        (
        #C11 & C12
            (lambda M : M[0] + M[1]),
            (lambda M : M[2] + M[3]),
        ),(
        #C21 & C22
            (lambda M : M[4] + M[5]),
            (lambda M : M[6] + M[7]),
        )
    ]
    def naive_mult_operations(n, t):
        # Each split increases the number of operations by a factor of 8, and we split log2(n/t) times
        return 8 ** np.log2(n / t) * (t ** 3)
    
    return BlockMMultFactory(scatter_targets, gather_tasks, threshold, max_thread_depth, naive_mult_operations)


n = 4
threshold = 1
max_thread_depth = 2

Strassen_Mult_Operator = create_strassen_callable(threshold, max_thread_depth)
Naive_Block_Mult_Operator = create_naive_callable(threshold, max_thread_depth)

A = np.random.rand(n,n)
B = np.random.rand(n,n)

C_Strassen = Strassen_Mult_Operator(A, B, verbose=2)
C_Naive = Naive_Block_Mult_Operator(A, B, verbose=2)


print("Result of Strassen Matrix Multiplication:\n", C_Strassen)
print("Result of Naive Matrix Multiplication:\n", C_Naive)

print("Expected result:\n", np.dot(A,B))

assert np.allclose(C_Strassen,np.dot(A,B))
assert np.allclose(C_Naive,np.dot(A,B))

IntProgress(value=0, max=49)

Total multiplications: 49


IntProgress(value=0, max=64)

Total multiplications: 64
Result of Strassen Matrix Multiplication:
 [[1.51175841 2.0157496  1.02343548 1.74030283]
 [1.65302182 2.01908324 0.95384942 1.96904066]
 [1.45442313 1.73306141 0.85300701 1.95457419]
 [1.19714278 1.59972217 0.84710902 1.21550315]]
Result of Naive Matrix Multiplication:
 [[1.51175841 2.0157496  1.02343548 1.74030283]
 [1.65302182 2.01908324 0.95384942 1.96904066]
 [1.45442313 1.73306141 0.85300701 1.95457419]
 [1.19714278 1.59972217 0.84710902 1.21550315]]
Expected result:
 [[1.51175841 2.0157496  1.02343548 1.74030283]
 [1.65302182 2.01908324 0.95384942 1.96904066]
 [1.45442313 1.73306141 0.85300701 1.95457419]
 [1.19714278 1.59972217 0.84710902 1.21550315]]


## Performance Benchmarking

Here we compare the performance between the two algorithms in terms of time, number of multiplications, and error.

In [ ]:
import timeit
import pandas as pd

def compare_errors(size, num_trials):
    errors_naive = []
    errors_strassen = []

    for _ in range(num_trials):
        A = generate_random_matrix(size)
        B = generate_random_matrix(size)

        # Standard numpy dot product
        result_dot = np.dot(A, B)

        # Naive matrix multiplication
        result_naive = naive_matrix_multiplication(A, B)
        error_naive = np.linalg.norm(result_naive - result_dot)

        # Strassen's matrix multiplication
        result_strassen = strassen_matrix_multiplication(A, B)  # Implement this function
        error_strassen = np.linalg.norm(result_strassen - result_dot)

        errors_naive.append(error_naive)
        errors_strassen.append(error_strassen)

    return np.mean(errors_naive), np.mean(errors_strassen)


def measure_performance(size, num_trials, threshold=1, max_thread_depth=2, verbose=1):
    
    Strassen_Mult_Operator = create_strassen_callable(threshold, max_thread_depth)
    Naive_Block_Mult_Operator = create_naive_callable(threshold, max_thread_depth)

    strassen_times = []
    naiveblock_times = []
    strassen_counts = []
    naiveblock_counts = []
    strassen_errs = []
    naiveblock_errs = []
    
    for _ in range(num_trials):
        A = np.random.rand(size,size)
        B = np.random.rand(size,size)
        res_true = np.dot(A,B)
                
        # Strassen
        start_time = timeit.default_timer()
        res, count = Strassen_Mult_Operator(A, B, verbose=verbose, return_count=True)
        strassen_times.append(timeit.default_timer() - start_time)
        strassen_counts.append(count)
        strassen_errs.append(np.linalg.norm(res - res_true))
        
        # Naive
        start_time = timeit.default_timer()
        res, count = Naive_Block_Mult_Operator(A, B, verbose=verbose, return_count=True)
        naiveblock_times.append(timeit.default_timer() - start_time)
        naiveblock_counts.append(count)
        naiveblock_errs.append(np.linalg.norm(res - res_true))
        

    return np.mean(strassen_times), np.mean(naiveblock_times), np.mean(strassen_counts), np.mean(naiveblock_counts), np.mean(strassen_errs), np.mean(naiveblock_errs)

max_power_of_2_dim = 11
sizes = [2**n for n in range(2, max_power_of_2_dim+1)]
num_trials = 1
threshold = 1
max_thread_depth = 3
verbose = 1

data = []
for size in sizes:
    print('measuring performance for mult on matrices of size ', size)
    performance = measure_performance(size, num_trials, threshold, max_thread_depth, verbose)
    data.append([size, *performance])

df = pd.DataFrame(data, columns=['Matrix Size', 'Strassen Time', 'Naive Time', 'Strassen Multiplications', 'Naive Multiplications', 'Strassen Error', 'Naive Error'])

measuring performance for mult on matrices of size  4


IntProgress(value=0, max=49)

IntProgress(value=0, max=64)

measuring performance for mult on matrices of size  8


IntProgress(value=0, max=343)

...

measuring performance for mult on matrices of size  128


IntProgress(value=0, max=823543)

IntProgress(value=0, max=2097152)

measuring performance for mult on matrices of size  256


IntProgress(value=0, max=5764801)

In [ ]:
from IPython.display import display, HTML

display(HTML(df.to_html()))

## Derivation

### step 1
Start by reformulating the problem of multiplying two $2 \times 2$ matrices into a matrix-vector product problem. This step abstracts the matrix multiplication into a more linear algebra-focused perspective, aiding in the simplification and manipulation of terms.

Given two $2 \times 2$ matrices:
$$
A = \begin{pmatrix} a & b \\ c & d \end{pmatrix}, \quad B = \begin{pmatrix} e & f \\ g & h \end{pmatrix}
$$

The objective is to compute the product \(C = AB\). We reshape this multiplication problem into a matrix-vector product by reorganizing the elements of \(A\) and \(B\) into new configurations that expose their linear interactions more clearly:
$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix}
\times
\begin{pmatrix} e \\ f \\ g \\ h \end{pmatrix}
$$

### Step 2: Decompose and Rearrange
Seek a decomposition of the \(4 \times 4\) matrix where each component (let's call these \(M_i\)) involves simpler, potentially rank-one matrices that are easier to handle. The goal is to express the large matrix as a sum of products where each product involves fewer independent variables. This part involves creative insight to "guess" beneficial decompositions.

For example, decompose the matrix into parts where each contributes separately to the output matrix with minimal overlap in variables used:
$$
\begin{pmatrix}
a & 0 & b & 0 \\
0 & a & 0 & b \\
c & 0 & d & 0 \\
0 & c & 0 & d
\end{pmatrix}
\rightarrow \text{Decomposed into several simpler matrices}
$$

### Step 3: Identify Rank-One Components
Strive to represent these decomposed matrices as sums of rank-one matrices (matrices that can be written as the outer product of two vectors). This step is crucial as it simplifies matrix multiplication to scalar multiplications, drastically reducing computational complexity.

### Step 4: Apply Linear Combinations
For each decomposed matrix, use linear combinations of the elements $a, b, c, d$ and $e, f, g, h$ to form new matrices $M_i$ whose sum recaptures the original $4 \times 4$ matrix. Each $M_i$ corresponds to a specific manipulation of the elements aimed at isolating particular interactions between $A$ and $B$.

### Step 5: Reassembly
Reconstruct the product matrix $C$ by summing the contributions from each $M_i$ applied to the vector of $B$'s components. The goal is to ensure that the collective effect of these simplified products gives the correct entries of $C$.

(from Yuval, 1978)

## Discussion

Although Strassen's algorithm clearly outperformed the Naive Block multiplication, Strassen's algorithm is not often used in practice. Better methods exist for sparse matrices, and most matrix multiplication occurs in the smaller dimensions where its greater overhead outweighs its increased error accumulation.

## References

Gideon Yuval,
A simple proof of Strassen's result,
Information Processing Letters,
Volume 7, Issue 6,
1978,
Pages 285-286,
ISSN 0020-0190,
https://doi.org/10.1016/0020-0190(78)90018-2.
(https://www.sciencedirect.com/science/article/pii/0020019078900182)
Keywords: Strassen's method; matrix multiplication; n2.81
    
    
https://cs.stackexchange.com/questions/130022/what-is-the-intuition-behind-strassens-algorithm